# Klassifikation

Du hast im letzten Teil schon gesehen, wie sich Daten von Reddit herunterladen lassen. Auch die Flairs hast du kennengelernt.

Allerdings hast du auch gesehen, dass nicht für alle Posts die Flairs richtig bereitgestellt werden:
* Es gibt alte Posts, bei denen die Flairs noch nicht eingeführt waren
* Nicht alle Autoren verwenden die Flairs konsistent

Mithilfe der Klassifikation hast du aber die Möglichkeite, auch die bisher nicht kategorisierten Daten noch den Flairs richtig zuzuordnen. Das wirst du in diesem Notebook durchführen.

## Achtung

Dieser Teil des Notebooks benötigt das gesamte Technology-Subreddit, das für den Download leider zu groß ist (> 10 GB). Daher ist dies im Colab-Notebook nicht ablauffähig! Eine Erklärung, wie du diese Daten selbst akquirieren kannst, findest du z.B. im iX-Artikel [Beziehungssache](https://www.heise.de/select/ix/2021/7/2102513144636338770). 

Du benötigst dies aber nur für die Erzeugung einer kleineren Datenmenge, die du dann anschließend im Notebook ausschließlich verwenden wirst und selbstverständlich dazu herunterladen kannst.

Bitte ab hier deshalb vorerst nur zuschauen!

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
sql = sqlite3.connect("technology.db")

Zunächst lädst du alle Posts ein. Viele Felder benötigst du nicht, um Speicher zu sparen, benutzt du nur die notwendigen.

In [ ]:
posts = pd.read_sql("SELECT id, url, created_utc, title, flair, parent_id, author, score, selftext, body FROM posts p", 
                  sql, parse_dates=["created_utc"])

Flairs wurden erst 2015 eingeführt und gelten nur für Toplevel-Posts. Du möchtest nur Posts verwenden, bei denen ein Flair gesetzt wurde, sonst könnten die Autoren das evtl. vergessen haben. Da du den `DataFrame` später noch modifizieren wirst, erzeugst du eine Kopie.

In [ ]:
top2015 = posts[(posts["created_utc"].dt.year>=2015) & 
                posts["parent_id"].isna() & 
                ~posts["flair"].isna()].copy()

Du bestimmst nun, ob die Posts zu Transportation gehören oder nicht. Beachte dabei, dass die Flairs *Transport* und *Transportation* genannt wurden.

In [ ]:
top2015["target"] = top2015["flair"].isin(["Transport", "Transportation"])

Wie verteilen sich die positiven und negativen Beispiele?

In [ ]:
top2015.value_counts("target")

Daraus erzeugst du ein stratifiziertes Datenset mit gleich vielen positiven wie negativen Beispielen: 

In [ ]:
pos = top2015[top2015["target"] == True]
neg = top2015[top2015["target"] == False]
data = pd.concat([pos, neg.sample(n = len(pos), random_state=42)], 
                 ignore_index=True)

Nun werden die Daten vektorisiert. Du verzichtest dabei auf Stopwords etc., die üblicherweise in der Textanalyse verwendet werden und vertraust auf die Funktionsweise von TF/IDF:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,1), max_df=0.7, min_df=5)
tfidf_vectors = tfidf.fit_transform(data["title"])

Als Konvention nennst du die unabhängige Variable `X` und die abhängige `y`:

In [ ]:
X = tfidf_vectors
y = data["target"].values

### Hold-out-Verfahren: Getrennte Mengen für Training und Test

Du teilst die Datenmenge in einen Teil, mit dem du den Klassifikator trainierst (75%) und eine, mit dem du die Ergebnisse verifizierst:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

Das Klassifikationsmodell wird als Support Vector Machine nur mit Trainingsdaten trainiert:

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
clf.fit(X_train, y_train)

Du führst eine Vorhersage für die (dem Klassifikator unbekannten) Testdaten durch:

In [ ]:
y_predicted = clf.predict(X_test)

Und betrachtest die Ergebnisse der Klassifikation:

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_predicted))

90% bzw. 96% Precision und Recall sind ziemlich gut, mit diesem Modell kannst du arbeiten und nun alle Posts (größtenteils richtig) klassifizieren:

## Gesamtes Datenset klassifizieren

Nun möchtest du die Trainingsdaten nutzen, um das gesamte oben eingelesene Datenset zu klassifizieren. Auch in den Kommentaren können nützliche Informationen enthalten sein, deswegen nutzt du die auch.

Dazu musst du zunächst den Text für alle Posts bestimmen. Dazu fügst du die einzelnen Textkomponenten mit Leerzeichen zusammen:

In [ ]:
posts["text"] = posts["title"].map(str) + " " + posts["body"].map(str) + " " + posts["selftext"].map(str)

Nun wendest du den Klassifikator an und klassifizierst alle Posts:

In [ ]:
posts["transport"] = clf.predict(tfidf.transform(posts["text"].map(str)))

Du interessierst dich für die Posts, die der Klassifikator als zu "Transport" zugehörig erkennt:

In [ ]:
transport = posts[posts["transport"] == True].copy()
transport

Die Daten speicherst du nun in einer CSV-Datei ab. Vorher ersetzt du noch die Umbrüche durch Leerzeichen, damit das CSV-File auch wieder richtig eingelesen werden kann.

In [ ]:
transport["text"] = transport["text"].str.replace("\n", " ")
transport["text"] = transport["text"].str.replace("\r", " ")

In [ ]:
transport.set_index("id")[["created_utc", "url", "parent_id", "author", "score", "text"]].\
          to_csv("transport-all-comments.csv", index_label="id")

## Zwischenergebnis

Nach all der Vorarbeit hast du  nun die Daten selektiert, mit denen du jetzt ausschließlich arbeiten wirst. 

Der Vorbereitungsaufwand mag hoch erscheinen, allerdings hast du nun auch wirklich ein Datenset, was genau zu deinen Geschäftsanforderungen passt. Mit manuellen Methoden wäre das nicht mit vertretbarem Aufwand möglich gewesen! 

Das ist auch ein Grund für die Beliebtheit von Data Science: mit relativ moderatem Aufwand können genau passende Datenmenge erzeugt werden!

Die Transport-Daten sind in einer SQLite-Datenbank abgespeichert, mit der du ab jetzt weiterarbeiten wirst. 